<a href="https://colab.research.google.com/github/Indhupamula/Resume-Screener-with-ATS-Score/blob/main/ai_powered_resume_screener.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!pip install gradio spacy nltk PyPDF2 python-docx docx2txt --quiet

# Download spaCy language model
!python -m spacy download en_core_web_sm

# Download NLTK stopwords
import nltk
nltk.download('stopwords')


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 18.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
import gradio as gr
import docx2txt
import PyPDF2
import os

def extract_text_from_file(file):
    file_path = file.name
    text = ""

    if file_path.endswith(".pdf"):
        reader = PyPDF2.PdfReader(file_path)
        for page in reader.pages:
            text += page.extract_text()
    elif file_path.endswith(".docx"):
        text = docx2txt.process(file_path)
    elif file_path.endswith(".txt"):
        with open(file_path, "r", encoding="utf-8") as f:
            text = f.read()
    else:
        return " Unsupported file format"

    return text

# Temporary function to preview extracted text
def preview_extraction(resume_file, job_description):
    resume_text = extract_text_from_file(resume_file)
    return f" Resume Text Extracted (First 500 chars):\n\n{resume_text[:500]}\n\n📝 Job Description:\n{job_description}"

demo = gr.Interface(
    fn=preview_extraction,
    inputs=[
        gr.File(label="Upload Resume (.pdf / .docx / .txt)"),
        gr.Textbox(lines=10, label="Paste Job Description")
    ],
    outputs="text",
    title=" Resume & Job Description Text Extractor (Preview)"
)

demo.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a73f063e822020e94d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [12]:
import spacy
from nltk.corpus import stopwords
import re

# Load NLP model and stopwords
nlp = spacy.load("en_core_web_sm")
stop_words = set(stopwords.words("english"))

def clean_and_extract_skills(text):
    # Lowercase and remove special characters
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text.lower())
    doc = nlp(text)
    keywords = set()

    for token in doc:
        if token.text not in stop_words and token.is_alpha and len(token.text) > 2:
            keywords.add(token.lemma_)

    return keywords

def calculate_match(resume_file, job_description):
    # Extract resume text
    resume_text = extract_text_from_file(resume_file)

    # Get keyword sets
    resume_skills = clean_and_extract_skills(resume_text)
    jd_skills = clean_and_extract_skills(job_description)

    # Match logic
    matched = resume_skills & jd_skills
    unmatched = jd_skills - resume_skills

    # Match score
    match_score = (len(matched) / len(jd_skills)) * 100 if jd_skills else 0

    # 🟢 FORMAT OUTPUT:
    result = f" ATS Match Score: {match_score:.2f}%\n\n"

    result += " Matched Skills:\n"
    result += ", ".join(sorted(matched)) if matched else "None"
    result += "\n\n"

    result += " Missing Skills:\n"
    result += ", ".join(sorted(unmatched)) if unmatched else "None"

    return result



# Gradio UI
demo = gr.Interface(
    fn=calculate_match,
    inputs=[
        gr.File(label="Upload Resume (.pdf / .docx / .txt)"),
        gr.Textbox(lines=10, label="Paste Job Description")
    ],
    outputs="text",
    title=" AI Resume Screener – ATS Match Score Calculator",
    description="Upload your resume and a job description to see how well they match."
)

demo.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://51651886f723f72a13.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [13]:
from huggingface_hub import notebook_login
notebook_login()


In [20]:
from huggingface_hub import HfApi

api = HfApi()


repo_id = "indhupamula/ai-resume-screener"

# Upload app.py
api.upload_file(
    path_or_fileobj="/content/ResumeParserApp/app.py",
    path_in_repo="app.py",
    repo_id=repo_id,
    repo_type="space"
)

# Upload fixed requirements.txt with spaCy model link
api.upload_file(
    path_or_fileobj="/content/ResumeParserApp/requirements.txt",
    path_in_repo="requirements.txt",
    repo_id=repo_id,
    repo_type="space"
)


No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/spaces/indhupamula/ai-resume-screener/commit/424d8ea5f4795392dabc723b4ff109bb903e3d2e', commit_message='Upload requirements.txt with huggingface_hub', commit_description='', oid='424d8ea5f4795392dabc723b4ff109bb903e3d2e', pr_url=None, repo_url=RepoUrl('https://huggingface.co/spaces/indhupamula/ai-resume-screener', endpoint='https://huggingface.co', repo_type='space', repo_id='indhupamula/ai-resume-screener'), pr_revision=None, pr_num=None)